## TORCH.JIT.TRACE

<b>
    
```python
torch.jit.trace(func, example_inputs, optimize=None, check_trace=True, check_inputs=None, check_tolerance=1e-05, strict=True, _force_outplace=False, _module_class=None, _compilation_unit=<torch.jit.CompilationUnit object>)
```
</b>

## Trace a Function and return an executable or ```ScriptModule``` that will be optimized using ```jit``` compilation.
Tracing is ideal for code that operates only on ```Tensor```s and lists, dictionaries and tuples ```Tensor```.

Using <i>```torch.jit.trace```</i> and <i>```torch.jit.trace_module```</i>, you can turn an existing module or Python function into TorchScript ```ScriptFunction``` or ```ScriptModule```. 

You must provide example inputs, run the function, recording the operations performed on all the tensors.
- The resulting recording of a standalone function produces <i>ScriptFunction</i>.
- The resulting recording of ```nn.Module.forward``` or ```nn.Module``` produces <i>ScriptModule</i>

### Parameters
1. <b>func</b>:- A Python Function or ```torch.nn.Module``` that will run with <i>example_inputs</i>. <i>func</i> arguments and return values must be tensors or (possibly nested) tuples that contain tensors. When a module is passed ```torch.jit.trace```, only the forward method is run and traced.

2. <b>example_inputs</b>:- A tuple of example inputs that will be passed to the function while tracing. The resulting trace can be run with input of different types and shapes assuming the traced operations support those types and shapes.

### Returns

If <i>func</i> is ```nn.Module``` or ```forward``` of <i>nn.Module</i>, returns a ```ScriptModule``` object with a single ```forward``` method containing the traced code.

The returned ```ScriptModule``` will have the same set of sub-modules and parameters as the original ```nn.Module```.
If ```func``` is a standalone function, ```trace``` returns ```ScriptFunction```.

### Example (Tracing a Function)

In [2]:
import torch

def foo(x, y):
    return 2 * x + y

# Run `foo` with provided inputs and record the tensor operations
traced_foo = torch.jit.trace(foo, (torch.rand(3), torch.rand(3)))

traced_foo

### Example (Tracing a Existing Module)

In [12]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(1, 1, 3)
        
    def forward(self, x):
        return self.conv(x)

model = Net()
weight = torch.rand(1, 1, 3, 3)
forward_input = torch.rand(1, 1, 3, 3)

print('Model: ', model)
print('\n')
# Trace a specific method and construct `ScriptModule` with
# a single `forward` method
module = torch.jit.trace(model.forward, forward_input)
print('Specific Method: ', module.code)
print(module.graph)

# Trace a module (implicitly traces `forward`) and construct a
# `ScriptModule` with a single `forward` method
module = torch.jit.trace(model, forward_input)
print(module)
print('Graph: ', module.graph)

Model:  Net(
  (conv): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
)


Specific Method:  def forward(self,
    x: Tensor) -> Tensor:
  return (self.conv).forward(x, )

graph(%self.1 : __torch__.___torch_mangle_66.Net,
      %x : Float(1, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cpu)):
  %31 : __torch__.torch.nn.modules.conv.___torch_mangle_65.Conv2d = prim::GetAttr[name="conv"](%self.1)
  %33 : Tensor = prim::CallMethod[name="forward"](%31, %x)
  return (%33)

Net(
  original_name=Net
  (conv): Conv2d(original_name=Conv2d)
)
Graph:  graph(%self.1 : __torch__.___torch_mangle_70.Net,
      %x : Float(1, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cpu)):
  %31 : __torch__.torch.nn.modules.conv.___torch_mangle_69.Conv2d = prim::GetAttr[name="conv"](%self.1)
  %33 : Tensor = prim::CallMethod[name="forward"](%31, %x)
  return (%33)

